In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import os
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Create dedicated directory for XGBoost model outputs
model_dir = 'xgboost_model'
os.makedirs(f'{model_dir}/plots', exist_ok=True)
os.makedirs(f'{model_dir}/data', exist_ok=True)

print("=" * 80)
print("XGBOOST MODEL FOR COMPLEX FEATURE INTERACTIONS IN SOLAR POWER PREDICTION")
print("=" * 80)

# 1. Load the processed data
print("\n1. Loading processed data...")
train_data = pd.read_csv('processed_data/train_all_predict_one/train_data.csv')
test_data = pd.read_csv('processed_data/train_all_predict_one/test_data.csv')

# Convert datetime for easier analysis
train_data['LocalTime'] = pd.to_datetime(train_data['LocalTime'])
test_data['LocalTime'] = pd.to_datetime(test_data['LocalTime'])

# 2. Create feature sets focused on XGBoost strengths
print("\n2. Creating XGBoost-specific features...")

def create_xgboost_features(data):
    """Create features optimized for XGBoost focusing on weather variables and interactions"""
    features = pd.DataFrame()
    
    # Numeric weather features
    weather_cols = [
        'Temperature', 'Dewpoint', 'Pressure', 'WindSpeed', 
        'WindDirection', 'GHI', 'DNI', 'DHI', 'Cloud_Coverage'
    ]
    
    # Check if columns exist and add them
    for col in weather_cols:
        if col in data.columns:
            features[col] = data[col]
        else:
            print(f"Warning: {col} not found in data columns. Using zero values.")
            features[col] = 0
    
    # Feature interactions
    if 'Temperature' in data.columns and 'GHI' in data.columns:
        features['temp_ghi'] = data['Temperature'] * data['GHI']
    
    if 'WindSpeed' in data.columns and 'GHI' in data.columns:
        features['wind_ghi'] = data['WindSpeed'] * data['GHI']
    
    if 'Cloud_Coverage' in data.columns and 'GHI' in data.columns:
        features['cloud_ghi'] = (100 - data['Cloud_Coverage']) * data['GHI'] / 100
    
    # Gradient features
    for col in ['Temperature', 'GHI', 'Cloud_Coverage']:
        if col in data.columns:
            try:
                features[f'{col}_diff'] = data.groupby('location_id')[col].diff().fillna(0)
            except:
                print(f"Warning: Could not calculate gradient for {col}")
    
    # Add polynomial features for key variables
    for col in ['GHI', 'Temperature']:
        if col in data.columns:
            features[f'{col}_squared'] = data[col] ** 2
    
    # Include Capacity_MW if available
    if 'Capacity_MW' in data.columns:
        features['Capacity_MW'] = data['Capacity_MW']
    
    return features

# Create feature sets
X_train_xgb = create_xgboost_features(train_data)
X_test_xgb = create_xgboost_features(test_data)

# For XGBoost, we should standardize numeric features for better performance
print("\n3. Standardizing numeric features...")
numeric_cols = X_train_xgb.columns

print(f"Standardizing {len(numeric_cols)} numeric features")

# Standardize numeric features
scaler = StandardScaler()
X_train_xgb[numeric_cols] = scaler.fit_transform(X_train_xgb[numeric_cols])
X_test_xgb[numeric_cols] = scaler.transform(X_test_xgb[numeric_cols])

# Save scaler for future use
with open(f'{model_dir}/data/feature_scaler.pkl', 'wb') as f:
    import pickle
    pickle.dump(scaler, f)

# Target variable
y_train = train_data['Power(MW)']
y_test = test_data['Power(MW)']

# Generate night mask for test data
test_night_mask = ~test_data['LocalTime'].dt.hour.between(5, 21)

# 4. Train the XGBoost model
print("\n4. Training XGBoost model...")

# Create validation set
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train_xgb, y_train, test_size=0.2, random_state=42
)

# Create DMatrix objects (XGBoost's optimized data structure)
dtrain = xgb.DMatrix(X_train_final, label=y_train_final)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test_xgb, label=y_test)

# Model parameters optimized for complex feature interactions
params = {
    'objective': 'reg:squarederror',  # Regression task
    'max_depth': 8,                   # Deeper trees for complex interactions
    'eta': 0.05,                      # Learning rate
    'subsample': 0.8,                 # Prevents overfitting
    'colsample_bytree': 0.8,          # Prevents overfitting
    'min_child_weight': 3,            # Controls complexity
    'gamma': 0.1,                     # Minimum loss reduction for split
    'alpha': 0.1,                     # L1 regularization
    'lambda': 1.0,                    # L2 regularization
    'tree_method': 'hist',            # Faster training method
    'seed': 42                        # Reproducibility
}

# Number of boosting rounds with early stopping
num_boost_round = 1000
early_stopping_rounds = 50

print(f"Training with {num_boost_round} boosting rounds and early stopping of {early_stopping_rounds} rounds")

# Train model with early stopping
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dval, 'validation')],
    early_stopping_rounds=early_stopping_rounds,
    verbose_eval=100
)

# Save the model
model_path = f'{model_dir}/xgboost_solar_model.json'
model.save_model(model_path)
print(f"Model saved to {model_path}")

# 5. Evaluate the model
print("\n5. Evaluating model performance...")

# Make predictions
y_pred = model.predict(dtest)

# Apply night mask (zero production during night)
y_pred_masked = y_pred.copy()
y_pred_masked[test_night_mask] = 0

# Calculate metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred_masked))
mae = mean_absolute_error(y_test, y_pred_masked)
r2 = r2_score(y_test, y_pred_masked)

print(f"RMSE: {rmse:.4f} MW")
print(f"MAE: {mae:.4f} MW")
print(f"R²: {r2:.4f}")

# Calculate daytime-only metrics
day_mask = ~test_night_mask
if np.any(day_mask):
    day_rmse = np.sqrt(mean_squared_error(y_test[day_mask], y_pred_masked[day_mask]))
    day_mae = mean_absolute_error(y_test[day_mask], y_pred_masked[day_mask])
    day_r2 = r2_score(y_test[day_mask], y_pred_masked[day_mask])
    print("\nDaytime-only metrics:")
    print(f"Daytime RMSE: {day_rmse:.4f} MW")
    print(f"Daytime MAE: {day_mae:.4f} MW")
    print(f"Daytime R²: {day_r2:.4f}")

# Save evaluation metrics to a file
with open(f'{model_dir}/evaluation_metrics.txt', 'w') as f:
    f.write(f"XGBoost Model Evaluation Metrics\n")
    f.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    f.write(f"RMSE: {rmse:.4f} MW\n")
    f.write(f"MAE: {mae:.4f} MW\n")
    f.write(f"R²: {r2:.4f}\n\n")
    
    if np.any(day_mask):
        f.write("Daytime-only metrics:\n")
        f.write(f"Daytime RMSE: {day_rmse:.4f} MW\n")
        f.write(f"Daytime MAE: {day_mae:.4f} MW\n")
        f.write(f"Daytime R²: {day_r2:.4f}\n")

# 6. Feature importance analysis
print("\n6. Analyzing feature importance...")

# Get feature importance from the model
importance_scores = model.get_score(importance_type='gain')
importance_df = pd.DataFrame({
    'Feature': list(importance_scores.keys()),
    'Importance': list(importance_scores.values())
}).sort_values('Importance', ascending=False)

# Plot feature importance
plt.figure(figsize=(24, 10))
plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.title('XGBoost Feature Importance (Gain)', fontsize=16)
plt.xlabel('Importance Score', fontsize=14)
plt.ylabel('Feature', fontsize=14)
plt.tight_layout()
plt.savefig(f'{model_dir}/plots/feature_importance.png', dpi=300, bbox_inches='tight')
plt.close()

# Save feature importance to CSV
importance_df.to_csv(f'{model_dir}/data/feature_importance.csv', index=False)

# Print top features
print("\nTop most important features:")
for i, (feature, importance) in enumerate(zip(importance_df['Feature'][:10], importance_df['Importance'][:10])):
    print(f"{i+1}. {feature}: {importance:.6f}")

# 7. SHAP Analysis for better feature interaction understanding
print("\n7. Performing SHAP analysis for feature interactions...")

try:
    # Use a smaller subset for SHAP analysis to manage memory
    shap_sample_size = min(500, X_test_xgb.shape[0])
    X_shap = X_test_xgb.iloc[:shap_sample_size]
    
    # Create explainer
    explainer = shap.TreeExplainer(model)
    
    # Calculate SHAP values
    print(f"Calculating SHAP values on {shap_sample_size} samples...")
    shap_values = explainer.shap_values(X_shap)
    
    # Plot summary
    plt.figure(figsize=(24, 10))
    shap.summary_plot(shap_values, X_shap, plot_type="bar", show=False)
    plt.title('Feature Impact on Solar Power Prediction (SHAP)', fontsize=16)
    plt.tight_layout()
    plt.savefig(f'{model_dir}/plots/shap_summary.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Create a detailed SHAP dependency plot for top feature
    if len(importance_df) > 0:
        top_feature = importance_df['Feature'].iloc[0]
        plt.figure(figsize=(24, 10))
        shap.dependence_plot(top_feature, shap_values, X_shap, show=False)
        plt.title(f'SHAP Dependence Plot for {top_feature}', fontsize=16)
        plt.tight_layout()
        plt.savefig(f'{model_dir}/plots/shap_dependence_plot.png', dpi=300, bbox_inches='tight')
        plt.close()
    
    # Save SHAP values for future analysis
    np.save(f'{model_dir}/data/shap_values.npy', shap_values)
    
    print("SHAP analysis completed and saved.")
    
except Exception as e:
    print(f"Error in SHAP analysis: {e}")
    print("Continuing without SHAP visualization...")

# 8. Create prediction visualizations 
print("\n8. Creating prediction visualizations...")

# Prepare data for plotting
test_results = pd.DataFrame({
    'LocalTime': test_data['LocalTime'],
    'Actual': y_test,
    'Predicted': y_pred_masked,
    'Hour': test_data['LocalTime'].dt.hour,
    'Date': test_data['LocalTime'].dt.date
})

# Save full results
test_results.to_csv(f'{model_dir}/data/predictions.csv', index=False)

# Calculate daily aggregated production
daily_results = test_results.groupby('Date').agg({
    'Actual': 'sum',
    'Predicted': 'sum'
}).reset_index()

# Save daily results
daily_results.to_csv(f'{model_dir}/data/daily_predictions.csv', index=False)

# Plot daily bar chart - wide format
plt.figure(figsize=(24, 10))
width = 0.35
x = np.arange(len(daily_results))
plt.bar(x - width/2, daily_results['Actual'], width, label='Actual', alpha=0.7)
plt.bar(x + width/2, daily_results['Predicted'], width, label='Predicted', alpha=0.7)

# Format x-axis with dates
plt.xticks(x, [d.strftime('%Y-%m-%d') for d in daily_results['Date']], rotation=45, fontsize=12)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Total Daily Power Production (MW)', fontsize=14)
plt.title('Daily Actual vs Predicted Solar Power Production (XGBoost)', fontsize=16)
plt.legend(fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{model_dir}/plots/daily_prediction.png', dpi=300, bbox_inches='tight')
plt.close()

# Plot hourly prediction for a sample day
# Select a day with good sunlight
sample_days = test_results['Date'].unique()
if len(sample_days) > 10:
    sample_day = sample_days[10]  # 10th test day
else:
    sample_day = sample_days[0]  # First available day
print(f"Creating hourly plot for example day: {sample_day}")

day_data = test_results[test_results['Date'] == sample_day]
plt.figure(figsize=(24, 10))
plt.plot(day_data['Hour'], day_data['Actual'], 'o-', label='Actual', linewidth=2, markersize=10)
plt.plot(day_data['Hour'], day_data['Predicted'], 's-', label='Predicted', linewidth=2, markersize=10)
plt.xlabel('Hour of Day', fontsize=14)
plt.ylabel('Power (MW)', fontsize=14)
plt.title(f'Hourly Solar Power Prediction for {sample_day} (XGBoost)', fontsize=16)
plt.xticks(range(0, 24), fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(fontsize=14)
plt.tight_layout()
plt.savefig(f'{model_dir}/plots/hourly_prediction.png', dpi=300, bbox_inches='tight')
plt.close()

# Create a heatmap of prediction accuracy by hour and month
print("\n9. Creating prediction accuracy heatmap by hour and month...")

# Add error metrics
test_results['AbsError'] = abs(test_results['Predicted'] - test_results['Actual'])
test_results['Month'] = test_results['LocalTime'].dt.month

# Create pivot table for heatmap
hour_month_error = test_results.pivot_table(
    values='AbsError', 
    index='Hour', 
    columns='Month', 
    aggfunc='mean'
)

# Plot heatmap
plt.figure(figsize=(24, 12))
sns.heatmap(hour_month_error, cmap='YlOrRd', annot=True, fmt='.2f')
plt.title('Mean Absolute Error by Hour and Month (XGBoost)', fontsize=16)
plt.xlabel('Month', fontsize=14)
plt.ylabel('Hour of Day', fontsize=14)
plt.tight_layout()
plt.savefig(f'{model_dir}/plots/error_heatmap.png', dpi=300, bbox_inches='tight')
plt.close()

# Create scatter plot of predicted vs actual
plt.figure(figsize=(16, 16))
plt.scatter(test_results['Actual'], test_results['Predicted'], alpha=0.5)
plt.plot([0, max(test_results['Actual'])], [0, max(test_results['Actual'])], 'r--')
plt.xlabel('Actual Power (MW)', fontsize=14)
plt.ylabel('Predicted Power (MW)', fontsize=14)
plt.title('XGBoost Predicted vs Actual Solar Power', fontsize=16)
plt.axis('equal')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{model_dir}/plots/prediction_scatter.png', dpi=300, bbox_inches='tight')
plt.close()

# 10. Analyze weather feature interactions
print("\n10. Analyzing weather feature interactions...")

try:
    # Find top two weather features
    weather_features = [f for f in importance_df['Feature'] if f in ['GHI', 'Temperature', 'DHI', 'Pressure']]
    if len(weather_features) >= 2:
        feat1 = weather_features[0]
        feat2 = weather_features[1]
        
        # Create a 3D visualization of the interaction
        from mpl_toolkits.mplot3d import Axes3D
        
        # Get the values for these features
        feat1_vals = X_test_xgb[feat1]
        feat2_vals = X_test_xgb[feat2]
        
        # Create 3D plot
        fig = plt.figure(figsize=(16, 12))
        ax = fig.add_subplot(111, projection='3d')
        
        # Scatter plot
        sc = ax.scatter(feat1_vals, feat2_vals, y_pred, c=y_pred, cmap='viridis', s=30, alpha=0.6)
        
        # Add labels
        ax.set_xlabel(feat1, fontsize=14)
        ax.set_ylabel(feat2, fontsize=14)
        ax.set_zlabel('Predicted Power Output (MW)', fontsize=14)
        ax.set_title(f'XGBoost Feature Interaction: {feat1} vs {feat2}', fontsize=16)
        
        # Add colorbar
        cbar = fig.colorbar(sc, ax=ax, pad=0.1)
        cbar.set_label('Predicted Power (MW)', fontsize=12)
        
        plt.tight_layout()
        plt.savefig(f'{model_dir}/plots/weather_interaction_3d.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"Created weather interaction plots for {feat1} and {feat2}")
except Exception as e:
    print(f"Error creating weather interaction plots: {e}")
    print("Continuing without interaction plots...")

print("\n11. Saving prediction results...")
# Already saved above, so just add a message

print("\nXGBoost model training and evaluation completed!")
print(f"This implementation focuses on XGBoost's strengths in modeling complex weather feature interactions")
print(f"Check the '{model_dir}' directory for all outputs:")
print(f"  - Model: {model_dir}/xgboost_solar_model.json")
print(f"  - Plots: {model_dir}/plots/")
print(f"  - Data: {model_dir}/data/")
print("=" * 80)

XGBOOST MODEL FOR COMPLEX FEATURE INTERACTIONS IN SOLAR POWER PREDICTION

1. Loading processed data...

2. Creating XGBoost-specific features...

3. Standardizing numeric features...
Standardizing 15 numeric features

4. Training XGBoost model...
Training with 1000 boosting rounds and early stopping of 50 rounds
[0]	validation-rmse:6.74844
[100]	validation-rmse:4.80228
[200]	validation-rmse:4.73822
[300]	validation-rmse:4.70220
[400]	validation-rmse:4.67632
[500]	validation-rmse:4.65755
[600]	validation-rmse:4.64119
[700]	validation-rmse:4.62725
[800]	validation-rmse:4.61688
[900]	validation-rmse:4.60784
[999]	validation-rmse:4.60183
Model saved to xgboost_model/xgboost_solar_model.json

5. Evaluating model performance...
RMSE: 4.8426 MW
MAE: 2.8623 MW
R²: 0.5276

Daytime-only metrics:
Daytime RMSE: 5.7539 MW
Daytime MAE: 4.0409 MW
Daytime R²: 0.4272

6. Analyzing feature importance...

Top most important features:
1. Temperature_diff: 1092.324829
2. GHI_squared: 526.526367
3. GHI: 347

<Figure size 2400x1000 with 0 Axes>